<a href="https://colab.research.google.com/github/Idaogah/datalab/blob/main/image_color_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Image Analysis and recognition
This project read images from different sources like webcam and Google drive, analyze the image and provide the color code and color name on the image

In [1]:
# !pip install --upgrade webcolors
!pip uninstall webcolors -y
!pip install webcolors==1.12

Found existing installation: webcolors 1.12
Uninstalling webcolors-1.12:
  Successfully uninstalled webcolors-1.12
  Using cached webcolors-1.12-py3-none-any.whl.metadata (2.0 kB)
Using cached webcolors-1.12-py3-none-any.whl (9.9 kB)


In [2]:

# Mount Drive
from google.colab import drive
drive.mount('/content/drive/')


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
import webcolors
import numpy as np
from PIL import Image

# Load the image
image_path = '/content/drive/My Drive/ML_AI/'
image = Image.open(image_path + 'text_color.jpg')

# Convert image to RGB
rgb_image = image.convert("RGB")

# Getting Text Color

In [14]:

# Get the pixel values at a region around the text
cropped_region = rgb_image.crop((30, 20, 200, 40))  # Adjusted coordinates based on image structure
cropped_pixels = np.array(cropped_region)

# Get the most common color in the cropped region
average_color = np.mean(cropped_pixels, axis=(0, 1))

# Convert the average color to both hex and RGB values
def rgb_to_hex_and_rgb(rgb):
    hex_value = "#{:02x}{:02x}{:02x}".format(int(rgb[0]), int(rgb[1]), int(rgb[2]))
    rgb_value = (int(rgb[0]), int(rgb[1]), int(rgb[2]))
    return hex_value, rgb_value

average_color_hex, average_color_rgb = rgb_to_hex_and_rgb(average_color)

# Function to get the closest color name
def closest_color(requested_color):
    min_colors = {}
    for key, name in webcolors.CSS3_HEX_TO_NAMES.items():
        r_c, g_c, b_c = webcolors.hex_to_rgb(key)
        rd = (r_c - requested_color[0]) ** 2
        gd = (g_c - requested_color[1]) ** 2
        bd = (b_c - requested_color[2]) ** 2
        min_colors[(rd + gd + bd)] = name
    return min_colors[min(min_colors.keys())]

# Try to find the exact or closest color name
def get_color_name(rgb_value):
    try:
        # Try to find the exact color name
        return webcolors.rgb_to_name(rgb_value)
    except ValueError:
        # If there's no exact match, return the closest color name
        return closest_color(rgb_value)

# Get the color name for the extracted RGB value
color_name = get_color_name(average_color_rgb)

# Output both the hex, RGB, and color name
average_color_hex, average_color_rgb, color_name

# Print the average color in hex format
print("The text color in the screenshot has a hex code of " + average_color_hex + \
      ".\n The closest color name is " + color_name + \
      ". \nThe RGB color code is " + str(average_color_rgb))

The text color in the screenshot has a hex code of #d9d9da.
 The closest color name is gainsboro. 
The RGB color code is (217, 217, 218)


# Getting Image Back ground color

In [26]:
# To find the background color, we'll sample a different region of the image that likely contains background pixels.
# Choosing a region slightly away from the text.

background_region = rgb_image.crop((0, 0, 20, 20))  # Coordinates adjusted to capture background
background_pixels = np.array(background_region)

# Get the most common color in the cropped region
average_background_color = np.mean(background_pixels, axis=(0, 1))

# Convert the average background color to a hex code
average_background_color_hex = rgb_to_hex_and_rgb(average_background_color)
color_name = get_color_name(average_background_color)

print("The background color of the screenshot is #" + color_name + \
      ",\n and Hex and RGB codes are" + str(average_background_color_hex))


The background color of the screenshot is #white,
 and Hex and RGB codes are('#ffffff', (255, 255, 255))


# Let's try another image

In [48]:
# Let's try the function on another image

image2 = Image.open(image_path + 'color_g.jpg')    # getting image from drive

hex_color, rgb_color = rgb_to_hex_and_rgb(np.mean(np.array(image2), axis=(0, 1)))     # analyze using the previous function
image2_name = get_color_name(rgb_color)

hex_color, rgb_color, image2_name

print("Color Name:", image2_name)
print("Hex Color", hex_color)
print("RGB Color", rgb_color)

Color Name: (256, 'yellowgreen')
Hex Color #00ef00
RGB Color (0, 239, 0)


# Let's capture image from Camera


In [42]:
import cv2  # For capturing images from the webcam

# Function to get an image from the webcam
def capture_image_from_camera():
    # Initialize the camera
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Could not open the camera")
        return None

    # Capture one frame
    ret, frame = cap.read()

    if not ret:
        print("Failed to capture image")
        return None

    # Release the camera
    cap.release()

    # Convert the captured frame (which is a numpy array) to PIL Image format
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image = Image.fromarray(frame_rgb)

    return image

# Main function
def main():
    # Capture an image from the camera
    image = capture_image_from_camera()
    if image is None:
        return
    image2 = Image.open(image_path + 'color_g.jpg')    # getting image from drive

    hex_color, rgb_color = rgb_to_hex_and_rgb(np.mean(np.array(image2), axis=(0, 1)))     # analyze using the previous function
    imgae2_name = get_color_name(rgb_color)

    hex_color, rgb_color, imgae2_name

    print("Color Name:", imgae2_name)
    print("Hex Color", hex_color)
    print("RGB Color", rgb_color)

# Run the main function
if __name__ == "__main__":
    main()


Could not open the camera


## Capture image from Camera using JavaScript

In [46]:
# Use JavaScript to capture an image
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from PIL import Image
import io

def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture Image';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize video to match its aspect ratio
      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      const ctx = canvas.getContext('2d');

      // Capture image when button is clicked
      await new Promise((resolve) => capture.onclick = resolve);

      ctx.drawImage(video, 0, 0);
      stream.getTracks().forEach(track => track.stop());
      div.remove();

      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(binary)
    return filename

# Take a photo
photo_filename = take_photo()



<IPython.core.display.Javascript object>

In [49]:
photo_filename = Image.open(image_path + 'color_g.jpg')    # getting image from drive

hex_color, photo_rgb_color = rgb_to_hex_and_rgb(np.mean(np.array(photo_filename), axis=(0, 1)))     # analyze using the previous function
photo_color_name = get_color_name(photo_rgb_color)

hex_color, rgb_color, photo_color_name

print("Color Name:", photo_color_name)
print("Hex Color", hex_color)
print("RGB Color", rgb_color)

Color Name: (256, 'yellowgreen')
Hex Color #00ef00
RGB Color (0, 239, 0)
